In [2]:
#前置作業準備，只要一次!!
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json

df5k = pd.read_json("paras/mapping5K.json")
dfv = pd.read_csv("volumeData/20230105.csv")
dfv["id"] = dfv["id"].astype("string")
dfv = dfv.drop(columns=['close'])

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36(KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36"
}

url4 = "https://www.wantgoo.com/investrue/all-alive"
r3 = requests.get(url4, headers = headers).content
soup = BeautifulSoup(r3, "html.parser")
rr3 = soup.prettify()
dfn = pd.read_json(rr3)
# dfn = pd.json_normalize(rr3, "industries" ,['id','name','shortName'])
dfn = dfn[(dfn["id"]>="1101") & (dfn["id"]<="9999") & (dfn["id"].str.len() == 4)]
dfn = dfn.drop(columns=['type','country','url', 'industries'])
dfn["id"] = dfn["id"].astype("string")
# dfn.info()

In [26]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
import _beowFmt as fm 
pd.set_option('display.expand_frame_repr', False)
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36(KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36"
}
url2 = "https://www.wantgoo.com/investrue/all-quote-info"

r = requests.get(url2, headers = headers).content
soup = BeautifulSoup(r, "html.parser")
rr = soup.prettify()
df = pd.read_json(rr)
df = df.drop(columns=['tradeDate', 'time','millionAmount'])
df["id"] = df["id"].astype("string")
df = pd.merge(df, dfn, left_on="id", right_on="id")  ## 結合股票名稱
df["float"] = df["close"].astype("float")
df["amp"] = (df["close"] - df["open"])/df["open"] * 100
df["jump"] = df["open"] - df["previousClose"]
df["jumpRate"] = df["jump"] / df["open"] * 100
df = df[(df["id"]>="1101") & (df["id"]<="9999") & (df["id"].str.len() == 4) & (df["close"] > df["previousClose"])].sort_values("jumpRate", ascending=False)
# df.info()
# print(df.head(50))

url3 = "https://www.wantgoo.com/stock/all-turnover-rates"
r2 = requests.get(url3, headers = headers).content
soup = BeautifulSoup(r2, "html.parser")
rr2 = soup.prettify()
dfa = pd.read_json(rr2)
# print(dfa)

dfa["investrueId"] = dfa["investrueId"].astype("string")
dfb = pd.merge(df, dfa, left_on="id", right_on="investrueId")
dfb = dfb.drop(columns=['investrueId'])
dfb = dfb[(dfb["value"] > 1)]

### 計算量比參數 ###
import time
localtime = time.localtime()
nowTime = time.strftime("%H:%M", localtime)
# print(nowTime)
# nowTime = "09:13"
Rate5k = df5k[(df5k["e"] >= nowTime) & (df5k["b"] <= nowTime)]["w"].values[0]  #五分K量比預估放大參數
print(f"目前時間：{nowTime},預估量放大係數:{Rate5k}")

dfc = pd.merge(dfb, dfv, left_on="id", right_on="id")
dfc["volumeEst"] = dfc["volume"] * Rate5k                               ##預估量
dfc["volumeDiff"] = dfc["volumeEst"] / dfc["yVolume"]
dfc2 = dfc[dfc["volumeDiff"] > 0]
print(dfc2.sort_values("value", ascending=False))

ss = ''
for d in dfc2["id"].tolist():
  ss += f"{d}.TW,"
fm.write_LogFile(f"xq_import/量價型態20230106.csv", ss)


######################################################################################################
# import _beowFmt as fm 
# dd = dfb[["id","close","volume"]].sort_values("id")
# ss = 'id,close,yVolume\n'
# for d in dd.values.tolist():
#   ss += f"{d[0]},{d[1]},{d[2]}\n"
# fm.write_LogFile(f"volumeData/20230105.csv", ss)
## todo ##
# https://www.wantgoo.com/investrue/all-alive       #股票名稱
# https://www.wantgoo.com/investrue/all-previous-weekly-candlestick?v=1672966860000  上一周收盤價


目前時間：01:18,預估量放大係數:1.0
      id   flat   floor    ceil    open   high     low   close  previousClose  volume name  market   float       amp  jump  jumpRate     value  yVolume  volumeEst  volumeDiff
14  1752   60.4   54.40   66.40   61.30   64.7   60.60   61.00           60.4   41920   南光  Listed   61.00 -0.489396  0.90  1.468189  41.28689    11533    41920.0    3.634787
4   8171   55.1   49.60   60.60   57.30   57.7   54.00   55.70           55.1   17822   天宇     OTC   55.70 -2.792321  2.20  3.839442  25.86356    21551    17822.0    0.826969
8   5321   75.0   67.50   82.50   76.90   79.0   75.30   75.70           75.0    6446  美而快     OTC   75.70 -1.560468  1.90  2.470741  15.72136     1474     6446.0    4.373134
6   3141   80.8   72.80   88.80   83.50   88.8   83.20   88.60           80.8   10934   晶宏     OTC   88.60  6.107784  2.70  3.233533  14.43629     1349    10934.0    8.105263
29  8996  179.0  161.50  196.50  180.00  186.0  180.00  184.00          179.0   13007   高力  Listed  18

In [30]:
import time
localtime = time.localtime()
nowTime = time.strftime("%H:%M", localtime)
print(nowTime)
# multiple_level_data = pd.json_normalize(data, record_path =['Results'], meta =['original_number_of_clusters','Scaler','family_min_samples_percentage'], meta_prefix='config_params_', record_prefix='dbscan_')

13:20


In [ ]:
import _beowFmt as fm 

df2 = df[(df["id"]>="1101") & (df["id"]<="9999") & (df["volume"] >= 1000)].sort_values(by=['id'], ascending=True)
dd = df2[{"id","volume"}]
ss = ''
for d in dd.values.tolist():
  ss += f"{d[1]}, {d[0]}\n"

fm.write_LogFile(f"keepData/stockVolume.csv", ss)

In [ ]:
while True:
    localtime = time.localtime()
    result = time.strftime("%Y-%m-%d %I:%M:%S %p", localtime)
    print(result)
    time.sleep(1)

In [ ]:
from pathlib import Path  

filepath = Path('https://www.wantgoo.com/investrue/all-quote-info')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath)  

In [ ]:
# importing the requests library
import requests

# api-endpoint
URL = "http://maps.googleapis.com/maps/api/geocode/json"

# location given here
location = "delhi technological university"

# defining a params dict for the parameters to be sent to the API
PARAMS = {'address':location}

# sending get request and saving the response as response object
r = requests.get(url = URL, params = PARAMS)

# extracting data in json format
data = r.json()


# extracting latitude, longitude and formatted address
# of the first matching location
latitude = data['results'][0]['geometry']['location']['lat']
longitude = data['results'][0]['geometry']['location']['lng']
formatted_address = data['results'][0]['formatted_address']

# printing the output
print("Latitude:%s\nLongitude:%s\nFormatted Address:%s"
	%(latitude, longitude,formatted_address))
